In [5]:
import pandas as pd
from generators import PatchGenerator, PatchSequence

In [15]:
def create_submisison(test_df, model_name, data_dir = './data', result_dir = './'):
    '''
    Function that generates a submission taking as an imput the corresponding test
    dataframe, the name of the best model to be loaded, the directory of the data
    and the directory of the model
    '''
    
    print('Evaluating test set ...')

    # load the best model
    model_path = join(result_dir, model_name)
    best_model = keras.models.load_model(model_path)

    # create a test sequence
    test_seq = PatchSequence(
        input_dir = './data', 
        dataframe = test_df, 
        batch_size = 32,
        dataset = 'test'
    )

    # run the model over the test samples
    y_pred = best_model.predict_generator(test_seq,
                                          use_multiprocessing=True,
                                          workers=4)
    
    # get the argument with the higher confidence score
    y_pred = np.argmax(y_pred, axis=1)
    
    # convert the indices in the correct labels
    y_true[y_true == 0] = 2
    y_true[y_true == 1] = 20
    y_true[y_true == 2] = 21

    # store the cases and corresponding class
    column_names = ['case', 'class']
    values = []
    for i in range(len(test_df)):
        case = test_seq.loc[i]['patientID'] + '_' + test_seq.loc[i]['studyID'] + '_' + test_seq.loc[i]['scanName']
        values.append(case, y_true[i])
    
    # create DataFrame with the values and export it to ".csv" format
    df = pd.DataFrame(data = values, columns = column_names)
    df.to_csv(join(result_dir, 'submission.csv'), index=None)